In [1]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup

from datetime import datetime
from tqdm import tqdm
import requests
import time
import glob

# Gorilla Chart 수집

In [98]:
# 1년치 데이터 수집
start_date='2019-01-01'
end_date='2024-03-01'
date_list=pd.date_range(start=start_date, end=end_date, freq='M')
date_list

DatetimeIndex(['2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30',
               '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31',
               '2019-09-30', '2019-10-31', '2019-11-30', '2019-12-31',
               '2020-01-31', '2020-02-29', '2020-03-31', '2020-04-30',
               '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31',
               '2020-09-30', '2020-10-31', '2020-11-30', '2020-12-31',
               '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30',
               '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31',
               '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31',
               '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
               '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31',
               '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31',
               '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-30',
               '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31',
      

## 1. 혜택별 Top10

```
   혜택별 숫자 매칭 
    dict = {'통신+공과금': [23,35],  
            '주유+차량정비': [21,118],  
            '쇼핑': [16],  
            '항공마일리지': [26],  
            '점심+교통': [5,137],  
            '무실적+모든가맹점': [11,12],  
            '구독/스트리밍': [169],  
            '해외직구': [27,151],  
            '배달앱+간편결제': [1,139],  
            '편의점+카페': [22,77,127],  
            '마트+교육비': [4,9,48,75],  
            '여행+바우처': [18,158],  
            '제휴/PLCC': [168],  
            '증권사CMA': [175]
            }
```

In [269]:
# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿 모드로 열기
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [305]:
rank = [i for i in range(1, 11)]

for date in date_list:
    get_date = date.strftime('%Y-%m-%d')
    # sub={혜택별 dict값 변경}
    drv.get(f'https://card-gorilla.com/chart/benefit?term=monthly&date={get_date}&sub=175')
    time.sleep(2.5)
    
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    ranks = soup.find_all(class_='rk_lst')
    
    name_list = []
    company_list = []
    href_list = []
    
    for i in range(len(ranks)):
        name_list.append(ranks[i].find(class_='card_name').text.strip())
        company_list.append(ranks[i].find(class_='corp_name').text)
        try:
            href_list.append('https://card-gorilla.com'+ranks[i].find('a')['href'])
        except:
            href_list.append('None')
            
    d_list = [get_date for i in range(10)]
    
    df = pd.DataFrame({'date':d_list, 'rank':rank, 'card_name':name_list, 
                       'card_company':company_list, 'card_href':href_list})
    
    if get_date == '2023-01-31':
        concat_df = df.copy()
    else:
        concat_df = pd.concat([concat_df, df], ignore_index=True)

drv.close()

In [308]:
concat_df

,date,rank,card_name,card_company,card_href
0,2023-01-31,1,부자되세요 더마일리지 체크카드,교보증권,https://card-gorilla.com/card/detail/1903
1,2023-01-31,2,able Premier Members 카드,KB증권,https://card-gorilla.com/card/detail/2225
2,2023-01-31,3,Win.K 체크카드,교보증권,https://card-gorilla.com/card/detail/1900
3,2023-01-31,4,able 카드 Ⅱ,KB증권,https://card-gorilla.com/card/detail/2219
4,2023-01-31,5,DB금융투자 해피플러스 체크카드,DB금융투자,https://card-gorilla.com/card/detail/2220
...,...,...,...,...,...
135,2024-02-29,6,미래에셋증권 체크카드 (캐시백 2),미래에셋증권,https://card-gorilla.com/card/detail/2217
136,2024-02-29,7,Win.K 체크카드,교보증권,https://card-gorilla.com/card/detail/1900
137,2024-02-29,8,한국투자 the More 체크카드,한국투자증권,https://card-gorilla.com/card/detail/2222
138,2024-02-29,9,able 아이맥스카드,KB증권,https://card-gorilla.com/card/detail/2223


In [309]:
# save
concat_df['promotion_cat'] = ['증권사CMA' for i in range(len(concat_df))]
concat_df.to_csv('project_data/증권사CMA_top10.csv', index=False, encoding='euc-kr')

## 2-1. 카드사별 Top10

```
    회사별 숫자 매칭
    dict = {'KB국민카드':3,
            '삼성카드':1,
            '롯데카드':4,
            'BC바로카드':32,
            '신한카드':2,
            '현대카드':7,
            '우리카드':5,
            'NH농협카드':9,
            '하나카드':8,
            'IBK기업은행':10
            }
```

In [158]:
# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿 모드로 열기
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [201]:
rank = [i for i in range(1, 11)]

for date in date_list:
    get_date = date.strftime('%Y-%m-%d')
    # sub={회사별 dict값 변경}
    drv.get(f'https://card-gorilla.com/chart/corp?term=monthly&date={get_date}&sub=10')
    time.sleep(5)
    
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    ranks = soup.find_all(class_='rk_lst')
    
    name_list = []
    company_list = []
    href_list = []
    
    for i in range(len(ranks)):
        name_list.append(ranks[i].find(class_='card_name').text.strip())
        company_list.append(ranks[i].find(class_='corp_name').text)
        try:
            href_list.append('https://card-gorilla.com'+ranks[i].find('a')['href'])
        except:
            href_list.append('None')
            
    d_list = [get_date for i in range(10)]
    
    df = pd.DataFrame({'date':d_list, 'rank':rank, 'card_name':name_list, 
                       'card_company':company_list, 'card_href':href_list})
    
    if get_date == '2023-01-31':
        concat_df = df.copy()
    else:
        concat_df = pd.concat([concat_df, df], ignore_index=True)
        
drv.close()

In [202]:
concat_df

,date,rank,card_name,card_company,card_href
0,2023-01-31,1,마일앤조이카드(대한항공),IBK기업은행,https://card-gorilla.com/card/detail/267
1,2023-01-31,2,BLISS.7 카드(마일리지),IBK기업은행,https://card-gorilla.com/card/detail/671
2,2023-01-31,3,ONE AIR(UniMile),IBK기업은행,https://card-gorilla.com/card/detail/475
3,2023-01-31,4,일상의 기쁨카드(신용),IBK기업은행,https://card-gorilla.com/card/detail/266
4,2023-01-31,5,마일앤조이카드(아시아나),IBK기업은행,https://card-gorilla.com/card/detail/268
...,...,...,...,...,...
135,2024-02-29,6,K-22(Mileage),IBK기업은행,https://card-gorilla.com/card/detail/2536
136,2024-02-29,7,마일앤조이카드(아시아나),IBK기업은행,https://card-gorilla.com/card/detail/268
137,2024-02-29,8,I-알뜰교통플러스 카드(신용),IBK기업은행,https://card-gorilla.com/card/detail/2559
138,2024-02-29,9,Daily With(데일리위드)카드,IBK기업은행,https://card-gorilla.com/card/detail/655


In [203]:
# save
concat_df.to_csv('project_data/IBK기업은행_top10.csv', index=False, encoding='euc-kr')

## 2-2. 카드별 혜택 정보 수집

In [169]:
df_list = glob.glob('project_data/*_top10.csv')
df_list

['project_data\\BC바로카드_top10.csv',
 'project_data\\IBK기업은행_top10.csv',
 'project_data\\KB국민카드_top10.csv',
 'project_data\\NH농협카드_top10.csv',
 'project_data\\롯데카드_top10.csv',
 'project_data\\삼성카드_top10.csv',
 'project_data\\신한카드_top10.csv',
 'project_data\\우리카드_top10.csv',
 'project_data\\하나카드_top10.csv',
 'project_data\\현대카드_top10.csv']

In [233]:
df = pd.read_csv(df_list[9], encoding='euc-kr')
df.drop_duplicates('card_href', inplace=True, ignore_index=True)
df = df[df['card_href'].isnull()==False]
df.reset_index(inplace=True, drop=True)
df

,date,rank,card_name,card_company,card_href
0,2023-01-31,1,현대카드ZERO Edition2(할인형),현대카드,https://card-gorilla.com/card/detail/608
1,2023-01-31,2,현대카드 M BOOST,현대카드,https://card-gorilla.com/card/detail/700
2,2023-01-31,3,네이버 현대카드,현대카드,https://card-gorilla.com/card/detail/2233
3,2023-01-31,4,American Express® Gold Card,현대카드,https://card-gorilla.com/card/detail/2281
4,2023-01-31,5,대한항공카드 030,현대카드,https://card-gorilla.com/card/detail/600
5,2023-01-31,6,the Green Edition2,현대카드,https://card-gorilla.com/card/detail/2260
6,2023-01-31,7,the Red Edition5,현대카드,https://card-gorilla.com/card/detail/2259
7,2023-01-31,8,현대카드Z family,현대카드,https://card-gorilla.com/card/detail/730
8,2023-01-31,9,현대카드ZERO Edition2(포인트형),현대카드,https://card-gorilla.com/card/detail/610
9,2023-01-31,10,현대카드 X BOOST,현대카드,https://card-gorilla.com/card/detail/701


In [26]:
df = pd.read_csv('project_data/credit_card_top100.csv')
df.drop_duplicates('card_href', inplace=True, ignore_index=True)
df = df[df['card_href'].isnull()==False]
df.reset_index(inplace=True, drop=True)
df

,date,rank,card_name,card_company,card_href
0,2023-01-31,1,LOCA 365 카드,롯데카드,https://card-gorilla.com/card/detail/2330
1,2023-01-31,2,DA@카드의정석,우리카드,https://card-gorilla.com/card/detail/87
2,2023-01-31,3,삼성카드 taptap O,삼성카드,https://card-gorilla.com/card/detail/51
3,2023-01-31,4,신한카드 Mr.Life,신한카드,https://card-gorilla.com/card/detail/13
4,2023-01-31,5,신한카드 Deep Dream,신한카드,https://card-gorilla.com/card/detail/32
...,...,...,...,...,...
168,2024-02-29,55,IBK무민카드(신용),IBK기업은행,https://card-gorilla.com/card/detail/723
169,2024-02-29,79,DA카드의정석 Ⅱ,우리카드,https://card-gorilla.com/card/detail/2655
170,2024-02-29,83,현대카드ZERO Edition3(할인형),현대카드,https://card-gorilla.com/card/detail/2646
171,2024-02-29,89,JADE Classic,하나카드,https://card-gorilla.com/card/detail/2657


In [27]:
# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿 모드로 열기
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [29]:
for index, href in tqdm(enumerate(df['card_href'])):
    drv.get(href)
    time.sleep(4)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    # event
    try:
        event = soup.find(class_='event_txt').text
    except:
        event = 'None'
    
    # promotion
    promotions_list = soup.find(class_='bnf1').find_all('dl')
    promotions = ''
    for i in range(len(promotions_list)):
        prm = promotions_list[i].find('dt').text + ' ' + promotions_list[i].find('dd').text
        prm.replace('최대', '').replace(',', '').strip().strip()
        promotions += prm + ' && '
    promotions = promotions[:-3].strip()
    
    # 연회비, 실적, 카드종류
    bottoms = soup.find(class_='bnf2').find_all('dl')
    
    fee = bottoms[0].find(class_='in_out').text.replace(',', '')
    
    perform = bottoms[1].text.replace('\n', '').replace('\t', '')
    
    brand_list = bottoms[2].find(class_='c_brand').find_all('span')
    brand = ''
    for text in brand_list:
        brand += text.text
        brand += ' '
    brand = brand.strip()
    
    dd = pd.DataFrame({'card_company': [df['card_company'][index]], 'card_name': [df['card_name'][index]], 'event': [event], 
                       'promotions': [promotions], 'fee': [fee], 'perform': [perform], 'brand': [brand]})
    
    if index==0:
        concat_df = dd
    else:
        concat_df = pd.concat([concat_df, dd], ignore_index=True)
    
    time.sleep(2)
    
drv.close()

173it [20:13,  7.02s/it]


In [30]:
concat_df

,card_company,card_name,event,promotions,fee,perform,brand
0,롯데카드,LOCA 365 카드,None,"아파트관리비, 공과금 10%할인 && 교통, 통신, 배달앱 10%할인 && 스트리밍...",국내전용 20000원 해외겸용 20000원,전월실적50만원 이상,mastercard AMEX
1,우리카드,DA@카드의정석,None,모든가맹점 0.8%할인 && 생활업종 1.3%할인 && 국내공항라운지 연 2회무료(...,국내전용 5000원 해외겸용 10000원,전월실적없음,BC mastercard
2,삼성카드,삼성카드 taptap O,최대 8만원 캐시백,"스타벅스 50%할인 && 대중교통 10%할인 && CGV, 롯데시네마 5,000원할인",국내전용 10000원 해외겸용 10000원,전월실적30만원 이상,mastercard
3,신한카드,신한카드 Mr.Life,신한카드 이용하고 최대 10만원 캐시백 받아가자!,"공과금 10%할인 && 마트,편의점 10%할인 && 식음료 10%할인",해외겸용 15000원,전월실적30만원 이상,VISA
4,신한카드,신한카드 Deep Dream,신한카드 이용하고 최대 10만원 캐시백 받아가자!,모두드림 0.7%적립 && 더해드림 2.1%적립 && 챙겨드림 3.5%적립,해외겸용 8000원,전월실적없음,VISA UnionPay
...,...,...,...,...,...,...,...
168,IBK기업은행,IBK무민카드(신용),카드고릴라 프로모션! 10만원 캐시백,"주요 커피전문점 20%할인 && 쿠팡, 티몬, 위메프 20%할인 && 편의점 10%할인",국내전용 10000원 해외겸용 10000원,전월실적50만원 이상,BC mastercard UnionPay
169,우리카드,DA카드의정석 Ⅱ,None,모든가맹점 0.8%할인 && 생활업종 1.3%할인 && 국내 공항라운지 무료이용,국내전용 12000원 해외겸용 12000원,전월실적없음,mastercard
170,현대카드,현대카드ZERO Edition3(할인형),신규회원 연회비 캐시백 이벤트,국내외 가맹점 0.8%할인,국내전용 15000원 해외겸용 15000원,전월실적없음,VISA AMEX
171,하나카드,JADE Classic,None,바우처 최대 10만원제공 && 전세계 공항라운지 연 3회 무료이용 && 국내외 가맹...,국내전용 115000원 해외겸용 120000원,전월실적없음,VISA


In [31]:
# save
concat_df.to_csv('project_data/credit_top100_info.csv', index=False)

In [236]:
# save
concat_df.to_csv('project_data/현대카드_info.csv', encoding='euc-kr', index=False)

## 3. 신용카드 Top100 수집

In [102]:
# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿 모드로 열기
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [104]:
rank = [i for i in range(1, 101)]

for date in tqdm(date_list):
    get_date = date.strftime('%Y-%m-%d')
    # sub={혜택별 dict값 변경}
    drv.get(f'https://www.card-gorilla.com/chart/top100?term=monthly&date={get_date}')
    time.sleep(7)
    
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    ranks = soup.find_all(class_='rk_lst')
    
    name_list = []
    company_list = []
    href_list = []
    
    for i in range(len(ranks)):
        name_list.append(ranks[i].find(class_='card_name').text.strip())
        company_list.append(ranks[i].find(class_='corp_name').text)
        try:
            href_list.append('https://card-gorilla.com'+ranks[i].find('a')['href'])
        except:
            href_list.append('None')
            
    d_list = [get_date for i in range(100)]
    
    df = pd.DataFrame({'date':d_list, 'rank':rank, 'card_name':name_list, 
                       'card_company':company_list, 'card_href':href_list})
    
    if get_date == '2019-01-31':
        concat_df = df.copy()
    else:
        concat_df = pd.concat([concat_df, df], ignore_index=True)
    time.sleep(2)

drv.close()

100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [10:13<00:00,  9.89s/it]


In [105]:
concat_df

,date,rank,card_name,card_company,card_href
0,2019-01-31,1,LIKIT FUN 카드,롯데카드,None
1,2019-01-31,2,삼성카드 taptap O,삼성카드,https://card-gorilla.com/card/detail/51
2,2019-01-31,3,신한카드 Deep Dream,신한카드,https://card-gorilla.com/card/detail/32
3,2019-01-31,4,NEW 씨티 클리어,씨티카드,https://card-gorilla.com/card/detail/101
4,2019-01-31,5,DA@카드의정석,우리카드,https://card-gorilla.com/card/detail/87
...,...,...,...,...,...
6195,2024-02-29,96,마일앤조이카드(아시아나),IBK기업은행,https://card-gorilla.com/card/detail/268
6196,2024-02-29,97,LOCA LIKIT Shop,롯데카드,https://card-gorilla.com/card/detail/2264
6197,2024-02-29,98,LOCA LIKIT Play,롯데카드,https://card-gorilla.com/card/detail/2263
6198,2024-02-29,99,American Express Blue,삼성카드,https://card-gorilla.com/card/detail/45


In [120]:
d = concat_df.drop_duplicates(['card_company', 'card_name'])
d['chart_in']=1
d

C:\Users\eunkk\AppData\Local\Temp\ipykernel_8644\2968069955.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['chart_in']=1


,date,rank,card_name,card_company,card_href,chart_in
0,2019-01-31,1,LIKIT FUN 카드,롯데카드,None,1
1,2019-01-31,2,삼성카드 taptap O,삼성카드,https://card-gorilla.com/card/detail/51,1
2,2019-01-31,3,신한카드 Deep Dream,신한카드,https://card-gorilla.com/card/detail/32,1
3,2019-01-31,4,NEW 씨티 클리어,씨티카드,https://card-gorilla.com/card/detail/101,1
4,2019-01-31,5,DA@카드의정석,우리카드,https://card-gorilla.com/card/detail/87,1
...,...,...,...,...,...,...
6151,2024-02-29,52,GOAT BC 바로카드,BC 바로카드,https://card-gorilla.com/card/detail/2644,1
6178,2024-02-29,79,DA카드의정석 Ⅱ,우리카드,https://card-gorilla.com/card/detail/2655,1
6182,2024-02-29,83,현대카드ZERO Edition3(할인형),현대카드,https://card-gorilla.com/card/detail/2646,1
6188,2024-02-29,89,JADE Classic,하나카드,https://card-gorilla.com/card/detail/2657,1


In [157]:
all_df.drop('chart_in', axis=1, inplace=True)

In [158]:
all_df = pd.merge(all_df, d[['card_company', 'card_name', 'chart_in']], on=['card_company', 'card_name'], how='outer')
idx = all_df[all_df['chart_in']!=1]['chart_in'].index
all_df.iloc[idx, -1] = 0

In [130]:
rank_mean = concat_df.groupby(['card_company', 'card_name'])[['rank']].agg('mean').reset_index()
all_df = pd.merge(df, rank_mean, on=['card_company', 'card_name'], how='outer')
idx = all_df[all_df['rank'].isnull()==True]['rank'].index
all_df.iloc[idx, -2] = 0

In [147]:
all_df['perform'] = [text.replace('전월실적', '').replace('만원 이상', '').replace('없음', '').strip() for text in all_df['perform']]
all_df['perform'] = all_df['perform'].replace('', 0)
all_df['perform'] = all_df['perform'].astype('int')

In [160]:
all_df['chart_in'].value_counts()

chart_in
0.0    617
1.0    370
Name: count, dtype: int64

In [165]:
idx = all_df[all_df['promotions'].isnull()==True].index
all_df.drop(idx, inplace=True)

In [166]:
all_df.to_csv('project_data/all_credit_info_4years.csv', encoding='utf-8', index=False)

In [106]:
# save
concat_df.to_csv('project_data/credit_card_top100_4year.csv', encoding='utf-8', index=False)

In [7]:
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6200 entries, 0 to 6199
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          6200 non-null   object
 1   rank          6200 non-null   int64 
 2   card_name     6200 non-null   object
 3   card_company  6200 non-null   object
 4   card_href     5159 non-null   object
dtypes: int64(1), object(4)
memory usage: 242.3+ KB


In [11]:
concat_df.describe(include='object')

,date,card_name,card_company,card_href
count,6200,6200,6200,5159
unique,62,370,14,306
top,2019-01-31,아시아나 신한카드 Air 1.5,신한카드,https://card-gorilla.com/card/detail/51
freq,100,62,899,62


## 파일 합치기

In [318]:
# 전체 카드 정보 및 순위 파일 합치기
# df_list = glob.glob('project_data/*_info.csv')
df_list = glob.glob('project_data/*_top10.csv')

for i in range(len(df_list)):
    if i == 0:
        all_df = pd.read_csv(df_list[i], encoding='euc-kr')
    if df_list[i] == 'project_data\\여행_바우처_top10.csv':
        all_df = pd.concat([all_df, pd.read_csv(df_list[i])], ignore_index=True)
    else:
        all_df = pd.concat([all_df, pd.read_csv(df_list[i], encoding='euc-kr')], ignore_index=True)

all_df.to_csv('project_data/card_promotion_top10_all.csv', index=False)

## 4. 전체 신용카드

### 4-1. href 수집

In [2]:
# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿 모드로 열기
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [3]:
drv.get(f'https://www.card-gorilla.com/search/card?cate=CRD')

In [16]:
for i in range(92):
    drv.find_element(By.CLASS_NAME, 'lst_more').click()
    time.sleep(2)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=124.0.6367.91)
Stacktrace:
	GetHandleVerifier [0x0033C113+48259]
	(No symbol) [0x002CCA41]
	(No symbol) [0x001C08A3]
	(No symbol) [0x00201A91]
	(No symbol) [0x001F7389]
	(No symbol) [0x00220D9C]
	(No symbol) [0x001F6E25]
	(No symbol) [0x00221034]
	(No symbol) [0x00239B9C]
	(No symbol) [0x00220B36]
	(No symbol) [0x001F570D]
	(No symbol) [0x001F62CD]
	GetHandleVerifier [0x005F6533+2908323]
	GetHandleVerifier [0x00633B4B+3159739]
	GetHandleVerifier [0x003D505B+674763]
	GetHandleVerifier [0x003DB21C+699788]
	(No symbol) [0x002D6244]
	(No symbol) [0x002D2298]
	(No symbol) [0x002D242C]
	(No symbol) [0x002C4BB0]
	BaseThreadInitThunk [0x75E07BA9+25]
	RtlInitializeExceptionChain [0x7729BE3B+107]
	RtlClearBits [0x7729BDBF+191]


In [18]:
soup = BeautifulSoup(drv.page_source, 'html.parser')
card_list = soup.find(class_='search_card').find(class_='con_area').find_all(class_='card-container')

In [27]:
card_name = []
card_company = []
href = []

for html in card_list:
    card_name.append(html.find(class_='card_name').text)
    card_company.append(html.find(class_='card_corp').text)
    href.append('https://www.card-gorilla.com'+html.find(class_='b_view')['href'])

In [32]:
all_data = pd.DataFrame({'card_name':card_name, 'card_company':card_company, 'href':href})
all_data.to_csv('project_data/all_card_href.csv', index=False)

### 4-2. 카드 정보 수집

In [17]:
df = pd.read_csv('project_data/all_card_href.csv')
df.drop_duplicates('href', inplace=True, ignore_index=True)
df = df[df['href'].isnull()==False]
df.reset_index(inplace=True, drop=True)
df.head(3), df.tail(3)

(                         card_name card_company  \
 0                     신한카드 Mr.Life         신한카드   
 1                 KB국민 My WE:SH 카드       KB국민카드   
 2  삼성카드 & MILEAGE PLATINUM (스카이패스)         삼성카드   
 
                                             href  
 0    https://www.card-gorilla.com/card/detail/13  
 1  https://www.card-gorilla.com/card/detail/2441  
 2    https://www.card-gorilla.com/card/detail/49  ,
                 card_name card_company  \
 920  현대카드ZERO MOBILE(할인형)         현대카드   
 921        현대카드ZERO(포인트형)         현대카드   
 922         현대카드ZERO(할인형)         현대카드   
 
                                              href  
 920  https://www.card-gorilla.com/card/detail/419  
 921  https://www.card-gorilla.com/card/detail/420  
 922  https://www.card-gorilla.com/card/detail/410  )

In [22]:
df.iloc[42]

card_name               American Express® Green Card Edition2
card_company                                             현대카드
href            https://www.card-gorilla.com/card/detail/2662
Name: 42, dtype: object

In [34]:
# 크롬창 띄우기
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') # 시크릿 모드로 열기
drv = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)

In [81]:
start = 800
end = 900

for index, href in tqdm(enumerate(df['href'][start:])):
    drv.get(href)
    time.sleep(4)
    soup = BeautifulSoup(drv.page_source, 'html.parser')
    
    tit = soup.find(class_='data_area').find(class_='tit')
    card_name = tit.find(class_='card').text
    card_company = tit.find(class_='brand').text

    # promotion
    promotions_list = soup.find(class_='bnf1').find_all('dl')
    promotions = ''
    for i in range(len(promotions_list)):
        prm = promotions_list[i].find('dt').text + ' ' + promotions_list[i].find('dd').text
        prm.replace('최대', '').replace(',', '').strip().strip()
        promotions += prm + ' && '
    promotions = promotions[:-3].strip()
    
    all_promotions = ''
    all_promotions_list = soup.find(class_='lst bene_area').find_all('dt')
    for i in range(len(all_promotions_list)):
        if all_promotions_list[i].find(class_='txt1').text == '유의사항':
            continue
        prm = all_promotions_list[i].find('i').text
        all_promotions += prm + ' && '
    all_promotions = all_promotions[:-3].strip()
    all_promotions
    
    # 연회비, 실적, 카드종류
    bottoms = soup.find(class_='bnf2').find_all('dl')
    fee = bottoms[0].find(class_='in_out').text.replace(',', '')
    perform = bottoms[1].text.replace('\n', '').replace('\t', '')
    try:
        brand_list = bottoms[2].find(class_='c_brand').find_all('span')
        brand = ''
        for text in brand_list:
            brand += text.text
            brand += ' '
        brand = brand.strip()
    except:
        brand = ''
    
    dd = pd.DataFrame({'card_company': [card_company], 'card_name': [card_name],
                       'promotions': [promotions], 'all_promotions': [all_promotions],
                       'fee': [fee], 'perform': [perform], 'brand': [brand]})
    
    if index==0:
        concat_df = dd
    else:
        concat_df = pd.concat([concat_df, dd], ignore_index=True)
        
    time.sleep(1)
    
drv.close()

123it [11:40,  5.70s/it]


In [83]:
# save
concat_df.to_csv(f'project_data/all_credit_info_{start}_{end}.csv', index=False, 
                 # encoding='euc-kr'
                )

In [85]:
df_list = glob.glob('project_data/all_credit_info_*.csv')
df_list

['project_data\\all_credit_info_0_100.csv',
 'project_data\\all_credit_info_100_200.csv',
 'project_data\\all_credit_info_200_300.csv',
 'project_data\\all_credit_info_300_400.csv',
 'project_data\\all_credit_info_400_500.csv',
 'project_data\\all_credit_info_500_600.csv',
 'project_data\\all_credit_info_600_700.csv',
 'project_data\\all_credit_info_700_800.csv',
 'project_data\\all_credit_info_800_900.csv']

In [87]:
df_list = glob.glob('project_data/all_credit_info_*.csv')

for i in range(len(df_list)):
    if i == 0:
        all_df = pd.read_csv(df_list[i])
    else:
        all_df = pd.concat([all_df, pd.read_csv(df_list[i])], ignore_index=True)

all_df.to_csv('project_data/all_credit_info.csv', index=False)

In [13]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   card_company    923 non-null    object
 1   card_name       923 non-null    object
 2   promotions      923 non-null    object
 3   all_promotions  921 non-null    object
 4   fee             923 non-null    object
 5   perform         922 non-null    object
 6   brand           887 non-null    object
dtypes: object(7)
memory usage: 50.6+ KB


In [14]:
all_df.describe(include='object')

,card_company,card_name,promotions,all_promotions,fee,perform,brand
count,923,923,923,921,923,922,887
unique,24,920,898,896,202,11,40
top,KB국민카드,어디로든 그린카드,위버스샵 4%적립 && 하이브인사이트 50%할인 && Gift서비스 연1회제공,Weverse Shop 이용금액의 4% Weverse Shop 캐시 추가 적립 &&...,국내전용 15000원 해외겸용 15000원,전월실적없음,Mastercard
freq,130,3,4,4,83,322,254
